In [ ]:
!pip install openai pandas


import os
import openai
import pandas as pd

# Set up your OpenAI API key
client = openai.OpenAI(api_key="**")


In [42]:
investor_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vertex_A1/dataset/InvestorScoreMatchingData.csv").head(10)
investor_df

,Name,Countries,Stage,Type,Cheque_min,Cheque_max,Industry
0,01 Ventures,"United Kingdom,Netherlands","pre-seed, idea, prototype/mvp, seed",VC,250000,2000000,IT
1,1 4 All Group,"DRC Congo,Ethiopia,Kenya,South Africa,Brazil,C...","pre-seed, seed, series+",Family office,100000,14000000,"Energy, IoT, IT, Healthcare, Consumer, Fintech..."
2,10D,Israel,"pre-seed, idea, prototype/mvp, seed",VC,500000,5000000,IT
3,10K Ventures,"Kenya,Nigeria,Brazil,Colombia,Mexico,India,Ind...","pre-seed, idea, prototype/mvp, seed",Family office,100000,200000,Sector Agnostic
4,10x Founders,"Germany,Austria,Switzerland,France,United King...","pre-seed, idea, prototype/mvp, seed",VC,100000,3000000,IT
5,11 Tribes Ventures,United States,"pre-seed, idea, prototype/mvp, seed, series+",VC,100000,300000,Sector Agnostic
6,1337 Ventures,"Malaysia,Vietnam,Philippines,Thailand","pre-seed, idea, prototype/mvp, seed, series+",VC,2000,50000,Sector Agnostic
7,13o3,"Bahrain,Australia,Austria,Belgium,Benin,Brazil...","pre-seed, idea, prototype/mvp, pre-ipo, seed, ...",PE fund,200000,1000000,"Energy, IoT, IT, Media, Fintech"
8,1Sharpe Ventures,"United States,Canada,Mexico,Brazil,United King...","pre-seed, idea, prototype/mvp, seed",VC,500000,2000000,Fintech
9,1Stock Venture Capital Fund,"Kenya,Nigeria,Rwanda,South Africa,Uganda","pre-seed, pre-ipo, seed, series+",VC,300000,2000000,IT


In [43]:
# Create a sample founder record
founder = {
    "Founder_Name": "John Doe",
    "Industry": "IT",            # Use lowercase to match investor data standardization
    "Startup_Stage": "seed",     # Example: seed, series a, etc.
    "Funding_Required": 500000,  # Funding required by the startup
    "Traction": "10K Users",     # Traction detail
    "Business_Model": "B2B"      # Business model type
}

print("Founder Profile:")
print(founder)

Founder Profile:
{'Founder_Name': 'John Doe', 'Industry': 'IT', 'Startup_Stage': 'seed', 'Funding_Required': 500000, 'Traction': '10K Users', 'Business_Model': 'B2B'}


In [44]:
def get_match_score(founder, investor):
    prompt = f"""
    You are an expert in startup-investor matchmaking. Given the details below, analyze the compatibility between a founder and an investor.
    Provide a match score between 0 and 100, where 100 means a perfect match, and include a short explanation for your score.

    Founder Details:
    - Industry: {founder['Industry']}
    - Startup Stage: {founder['Startup_Stage']}
    - Funding Required: {founder['Funding_Required']}
    - Traction: {founder['Traction']}
    - Business Model: {founder['Business_Model']}

    Investor Details:
    - Name: {investor['Name']}
    - Preferred Industry: {investor['Industry']}
    - Preferred Startup Stage(s): {investor['Stage']}
    - Investment Range: {investor['Cheque_min']} to {investor['Cheque_max']}

    Please provide the result in the following format:
    Score: <score>
    Explanation: <brief explanation>
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0
        )
        # Extract the response text using the new API format
        result = response.choices[0].message.content
        return result
    except Exception as e:
        return f"Error: {e}"

In [45]:
# Function to extract the score as a float from the API response text
def extract_score(response_text):
    # Ensure the input is a string
    if not isinstance(response_text, str):
        response_text = str(response_text)
    # Expecting response like: "Score: <number>\nExplanation: <...>"
    match = re.search(r"Score:\s*([\d\.]+)", response_text, re.IGNORECASE)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return 0.0
    return 0.0

In [46]:
# Process only the first 100 investors and build a list with investor name, score, and explanation
matched_investors = []
for index, investor in investor_df.iterrows():
    response_text = get_match_score(founder, investor)
    score = extract_score(response_text)
    matched_investors.append({
        "Investor Name": investor["Name"],
        "Match Score": score,
        "Details": response_text
    })

# Convert results to a DataFrame for better visualization and sort by score (descending)
result_df = pd.DataFrame(matched_investors)
result_df = result_df.sort_values(by="Match Score", ascending=False)


In [48]:
# Display the top 5 ranked investors
print("\nTop Ranked Investors:")
result_df.head()


Top Ranked Investors:


,Investor Name,Match Score,Details
0,01 Ventures,95.0,Score: 95\nExplanation: The founder and the in...
2,10D,95.0,Score: 95\nExplanation: The founder and the in...
9,1Stock Venture Capital Fund,90.0,Score: 90\n\nExplanation: The founder and the ...
4,10x Founders,90.0,Score: 90\nExplanation: The founder and the in...
7,13o3,90.0,Score: 90\n\nExplanation: The founder and inve...


In [49]:
founder

{'Founder_Name': 'John Doe',
 'Industry': 'IT',
 'Startup_Stage': 'seed',
 'Funding_Required': 500000,
 'Traction': '10K Users',
 'Business_Model': 'B2B'}